In [26]:
from flair.data import Corpus
from flair.data import Sentence
from typing import List
import torch
import pandas as pd
import numpy as np
import flair
from flair.embeddings import CamembertEmbeddings, FlairEmbeddings, TokenEmbeddings, WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence
from pathlib import Path

In [14]:
data = pd.read_csv('Training_dataset_phonenumber.csv', encoding='latin1')
data = data.fillna(method='ffill')
data.head(20)

,Sentence,Word,Tag
0,Sentence: 1,Vous,O
1,Sentence: 1,pouvez,O
2,Sentence: 1,me,O
3,Sentence: 1,contacter,O
4,Sentence: 1,sur,O
5,Sentence: 1,mon,O
6,Sentence: 1,numéro,O
7,Sentence: 1,portable,O
8,Sentence: 1,698768760,S-TEL
9,Sentence: 1,.,O


In [15]:
class Sentence_getter(object):
    def __init__(self, data):
        self.data = data
        agg_fun = lambda s: [(w, t) for w,t in zip(s["Word"].values.tolist(),
                                                       s['Tag'].values.tolist())]
        
        self.grouped = self.data.groupby('Sentence').apply(agg_fun)
        self.sentences = [i for i in self.grouped]

In [16]:
getter = Sentence_getter(data)
sentences = getter.sentences

In [17]:
sentences[0]

[('Vous', 'O'),
 ('pouvez', 'O'),
 ('me', 'O'),
 ('contacter', 'O'),
 ('sur', 'O'),
 ('mon', 'O'),
 ('numéro', 'O'),
 ('portable', 'O'),
 ('698768760', 'S-TEL'),
 ('.', 'O')]

In [18]:
tagged = list()
listOfSent =list()   
strr=""
for a in range(len(sentences)):

    for word in sentences[a] :
        strr = strr + word[0]+ " "
        tagged.append(word[1])
        
    sent = Sentence(strr)
    print(strr)
    print(sent)
    for x in range(len(sent)):
        sent[x].add_tag('ner', tagged[x])
        print('')
        
    strr =""
    tagged=[]  
    listOfSent.append(sent)

Vous pouvez me contacter sur mon numéro portable 698768760 . 
Sentence: "Vous pouvez me contacter sur mon numéro portable 698768760 ."   [− Tokens: 10]










Mon nouveau numéro portable est 6543456787 . 
Sentence: "Mon nouveau numéro portable est 6543456787 ."   [− Tokens: 7]







3454323456 est mon ancien numéro portable . 
Sentence: "3454323456 est mon ancien numéro portable ."   [− Tokens: 7]







Le numéro 76543456787 n n est pas correct . 
Sentence: "Le numéro 76543456787 n n est pas correct ."   [− Tokens: 9]









Le numéro 98765456787 n n est pas valide . 
Sentence: "Le numéro 98765456787 n n est pas valide ."   [− Tokens: 9]









Ce numéro 3434234567 est erroné . 
Sentence: "Ce numéro 3434234567 est erroné ."   [− Tokens: 6]






Le numéro auquel on devrait envoyer notre demande est 76545676543 . 
Sentence: "Le numéro auquel on devrait envoyer notre demande est 76545676543 ."   [− Tokens: 11]











Mon numéro portable est 003307653432123 . 
Sentence: "Mon 

In [19]:
train = listOfSent[:10]
dev = listOfSent[:2]
test= listOfSent[-5:]

In [20]:
corpus = Corpus(train, dev, test)

# Make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type="ner")
print(tag_dictionary.idx2item)

stacked_embeddings = StackedEmbeddings([
    TransformerWordEmbeddings('camembert-base')
                                       ])

# Initialize sequence tagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=stacked_embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type="ner",
                                        use_crf=True)

# initialize trainer
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# Start training
trainer.train('resources/taggers/flair+camembertEmbeddings',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=30,
              checkpoint=True)

[b'<unk>', b'O', b'S-TEL', b'<START>', b'<STOP>']
2020-12-22 20:12:00,646 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:00,649 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): CamembertModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(32005, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0): RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): L

2020-12-22 20:12:00,650 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:00,651 Corpus: "Corpus: 10 train + 2 dev + 5 test sentences"
2020-12-22 20:12:00,652 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:00,654 Parameters:
2020-12-22 20:12:00,655  - learning_rate: "0.1"
2020-12-22 20:12:00,656  - mini_batch_size: "32"
2020-12-22 20:12:00,656  - patience: "3"
2020-12-22 20:12:00,657  - anneal_factor: "0.5"
2020-12-22 20:12:00,658  - max_epochs: "30"
2020-12-22 20:12:00,658  - shuffle: "True"
2020-12-22 20:12:00,659  - train_with_dev: "False"
2020-12-22 20:12:00,660  - batch_growth_annealing: "False"
2020-12-22 20:12:00,661 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:00,661 Model training base path: "resources/taggers/flair+camembertEmbeddings"
2020-12-22 20:12:00,662 --------

2020-12-22 20:12:23,372 EPOCH 13 done: loss 0.3849 - lr 0.1000000
2020-12-22 20:12:23,443 DEV : loss 0.021999359130859375 - score 1.0
2020-12-22 20:12:23,444 BAD EPOCHS (no improvement): 0
saving best model
2020-12-22 20:12:25,144 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:25,474 epoch 14 - iter 1/1 - loss 0.08531799 - samples/sec: 97.43 - lr: 0.100000
2020-12-22 20:12:25,476 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:25,478 EPOCH 14 done: loss 0.0853 - lr 0.1000000
2020-12-22 20:12:25,532 DEV : loss 0.018497467041015625 - score 1.0
2020-12-22 20:12:25,533 BAD EPOCHS (no improvement): 0
saving best model
2020-12-22 20:12:27,039 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:27,317 epoch 15 - iter 1/1 - loss 0.11406364 - samples/sec: 116.31 - lr: 0.100000
2020-12-22 20:

2020-12-22 20:12:46,227 EPOCH 28 done: loss 0.2263 - lr 0.1000000
2020-12-22 20:12:46,266 DEV : loss 0.0047454833984375 - score 1.0
2020-12-22 20:12:46,266 BAD EPOCHS (no improvement): 0
saving best model
2020-12-22 20:12:47,434 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:47,684 epoch 29 - iter 1/1 - loss 0.17476597 - samples/sec: 128.63 - lr: 0.100000
2020-12-22 20:12:47,685 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:47,686 EPOCH 29 done: loss 0.1748 - lr 0.1000000
2020-12-22 20:12:47,723 DEV : loss 0.0045871734619140625 - score 1.0
2020-12-22 20:12:47,724 BAD EPOCHS (no improvement): 0
saving best model
2020-12-22 20:12:48,910 ----------------------------------------------------------------------------------------------------
2020-12-22 20:12:49,175 epoch 30 - iter 1/1 - loss 0.48194543 - samples/sec: 121.29 - lr: 0.100000
2020-12-22 20:

{'test_score': 0.9787,
 'dev_score_history': [0.7647,
  0.8824,
  0.7647,
  0.8824,
  1.0,
  0.8824,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'train_loss_history': [16.946102142333984,
  7.455700874328613,
  4.634989261627197,
  3.3824753761291504,
  3.5738914012908936,
  2.247823715209961,
  2.217273473739624,
  1.0744346380233765,
  1.1054118871688843,
  0.5948624610900879,
  1.0565366744995117,
  0.35323983430862427,
  0.3849243223667145,
  0.08531799167394638,
  0.11406364291906357,
  0.04297485202550888,
  0.514988124370575,
  0.31660404801368713,
  0.30119380354881287,
  0.08867206424474716,
  0.042867280542850494,
  0.46262073516845703,
  0.02562694624066353,
  0.08289756625890732,
  0.03618926927447319,
  0.04135780408978462,
  0.3986572325229645,
  0.2262779176235199,
  0.17476597428321838,
  0.4819454252719879],
 'dev_loss_history': [6.7027053833007

In [21]:
model = SequenceTagger.load('resources/taggers/flair+camembertEmbeddings/best-model.pt')

2020-12-22 20:12:52,994 loading file resources/taggers/flair+camembertEmbeddings/best-model.pt


In [22]:
sentence = Sentence('Le numéro de téléphone de Emma Louise est 0676765432. Elle était diplômée en 2019. Si ce numéro est injoignable, il faudra appeler 0612121212.')

In [23]:
%%time
model.predict(sentence)

CPU times: user 196 ms, sys: 21.4 ms, total: 217 ms
Wall time: 212 ms


In [24]:
print(sentence.to_tagged_string())

Le numéro de téléphone de Emma Louise est 0676765432 <S-TEL> . Elle était diplômée en 2019 . Si ce numéro est injoignable , il faudra appeler 0612121212 <S-TEL> .


In [33]:
#[a.b] Fix me: Combine Flair Corpus with existing model

In [27]:
checkpoint = 'resources/taggers/flair+camembertEmbeddings/checkpoint.pt'
trainer = ModelTrainer.load_checkpoint(checkpoint, flair.datasets.WIKINER_FRENCH().downsample(0.1))
trainer.train('resources/taggers/flair+camembertEmbeddings+flaircorpus',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=3,
              checkpoint=True)

2020-12-22 20:46:38,878 Reading data from /Users/amalbedoui/.flair/datasets/wikiner_french
2020-12-22 20:46:38,878 Train: /Users/amalbedoui/.flair/datasets/wikiner_french/aij-wikiner-fr-wp3.train
2020-12-22 20:46:38,879 Dev: None
2020-12-22 20:46:38,879 Test: None
2020-12-22 20:46:44,372 ----------------------------------------------------------------------------------------------------
2020-12-22 20:46:44,375 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): CamembertModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(32005, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (

2020-12-22 20:46:44,376 ----------------------------------------------------------------------------------------------------
2020-12-22 20:46:44,377 Corpus: "Corpus: 10713 train + 1190 dev + 1323 test sentences"
2020-12-22 20:46:44,377 ----------------------------------------------------------------------------------------------------
2020-12-22 20:46:44,378 Parameters:
2020-12-22 20:46:44,378  - learning_rate: "0.1"
2020-12-22 20:46:44,379  - mini_batch_size: "32"
2020-12-22 20:46:44,380  - patience: "3"
2020-12-22 20:46:44,380  - anneal_factor: "0.5"
2020-12-22 20:46:44,381  - max_epochs: "3"
2020-12-22 20:46:44,382  - shuffle: "True"
2020-12-22 20:46:44,382  - train_with_dev: "False"
2020-12-22 20:46:44,383  - batch_growth_annealing: "False"
2020-12-22 20:46:44,384 ----------------------------------------------------------------------------------------------------
2020-12-22 20:46:44,384 Model training base path: "resources/taggers/flair+camembertEmbeddings+flaircorpus"
2020-12-22 2

{'test_score': 0.8776,
 'dev_score_history': [],
 'train_loss_history': [],
 'dev_loss_history': []}

In [29]:
newmodel = SequenceTagger.load('resources/taggers/flair+camembertEmbeddings+flaircorpus/final-model.pt')

2020-12-22 20:55:42,658 loading file resources/taggers/flair+camembertEmbeddings+flaircorpus/final-model.pt


In [30]:
newsentence = Sentence('Emma Louise, habite au 26 rue Alexandre, 75005 Paris, France, née le 11/11/1993. Elle est F et travaille chez Zenika et elle est joignable sur 06660006.')

In [31]:
%%time
newmodel.predict(newsentence)

CPU times: user 188 ms, sys: 15.6 ms, total: 203 ms
Wall time: 198 ms


In [32]:
print(newsentence.to_tagged_string())

Emma Louise , habite au 26 <S-TEL> rue Alexandre , 75005 Paris , France , née le 11 <S-TEL> / 11 / 1993 . Elle est F et travaille chez Zenika et elle est joignable sur 06660006 <S-TEL> .
